In [12]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra, DecisionTree

In [13]:
data_surverse = CSV.read("data/surverses.csv",missingstring="-99999")
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [14]:
data_surverse = filter(row -> month(row.DATE) > 4, data_surverse) 
data_surverse = filter(row -> month(row.DATE) < 11, data_surverse) 
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [15]:
raison = coalesce.(data_surverse[:,:RAISON],"Inconnue")
data_surverse[!,:RAISON] = raison
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


In [16]:
data_surverse = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data_surverse) 
select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [17]:
surverse_df = dropmissing(data_surverse, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [18]:
data_precipitations = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data_precipitations, Symbol("St-Hubert")=>:StHubert)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


In [19]:
data_precipitations = filter(row -> month(row.date) > 4, data_precipitations) 
data_precipitations = filter(row -> month(row.date) < 11, data_precipitations) 
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


In [138]:
# Analyse exploratoire

# Faire la somme des precipitations par rapport a la date
pcp_sum = by(data_precipitations, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
           Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

pcp_max = by(data_precipitations, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

# Remplacer les valeurs manquantes par des 0
# raison = coalesce.(pcp_sum, 0)
# pcp_sum = raison

# raison = coalesce.(pcp_max, 0)
# pcp_max = raison

df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", surverse_df)

rename!(df_surve, Symbol("DATE")=>:date)

enhanced_surve = join(df_surve, pcp_sum, on=:date, kind=:inner)

for i = 1:size(enhanced_surve,1)
    # row average
    # non_missing_avg = sum(skipmissing(dfs_enhanced_surve[i][j,[4,5,6,7]])) / 4
    sum = 0
    for k = 1:5
        if enhanced_surve[i,k + 3] !== missing
            sum = sum + enhanced_surve[i,k + 3]
        end
    end
    for k = 1:5
        # plus 3 pour skip les 3 premieres colones
        if (enhanced_surve[i,k + 3] === missing)
            enhanced_surve[i,k + 3] = floor(sum/5)
        end
    end
end

train_data = convert(Matrix, enhanced_surve[:,[:McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]])
train_result = convert(Array{Float64}, enhanced_surve[:,:SURVERSE])

model = build_forest(train_result, train_data, 2, 200, 0.5, 32)


,NO_OUVRAGE,date,SURVERSE,McTavish,Bellevue,Assomption,Trudeau,StHubert
,String,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,3260-01D,2013-05-01,0,0,0,0,0,0
2,3260-01D,2013-05-02,0,0,0,0,0,0
3,3260-01D,2013-05-03,0,0,0,0,0,0
4,3260-01D,2013-05-04,0,0,0,0,0,0
5,3260-01D,2013-05-05,0,0,0,0,0,0
6,3260-01D,2013-05-06,0,0,0,0,0,0
7,3260-01D,2013-05-07,0,0,0,0,0,0
8,3260-01D,2013-05-08,0,0,0,0,0,0
9,3260-01D,2013-05-09,0,10,0,19,0,5


In [140]:
### Retrieve test data
test = CSV.read("data/test.csv");

test_df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", test)

rename!(test_df_surve, Symbol("DATE")=>:date) 

test_df_serve = join(test_df_surve, pcp_sum, on=:date, kind=:inner)

for i = 1:size(test_df_serve,1)
    # row average
    sum = 0
    for k = 1:5
        if test_df_serve[i,k + 2] !== missing
            sum = sum + test_df_serve[i,k + 2]
        end
    end
    for k = 1:5
        # plus 3 pour skip les 3 premieres colones
        if (test_df_serve[i,k + 2] === missing)
            test_df_serve[i,k + 2] = floor(sum/5)
        end
    end
end

test_data = convert(Matrix, test_df_serve[:,[:McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]])

n = size(test_df_serve, 1)
result = Array{Float64}(undef, n)

for i = 1:n
    result[i] = apply_forest(model, test_data[i, :])
end

Y = zeros(Int64, n)
Y[result.>.5] .= 1
Y

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [142]:
ID = Array{String}(undef, n)
for i = 1:n
    ID[i] = string(test[i,:NO_OUVRAGE], "_", Dates.format(test[i,:DATE], "yyyy-mm-dd"))
end
YTotalBool = Array{String}(undef, n)

for i = 1:n
    if Y[i] == 0
        YTotalBool[i] = "FALSE"
    else
        YTotalBool[i] = "TRUE"
    end
end

prediction = DataFrame(ID = ID, Surverse = YTotalBool)
CSV.write("sampleSubmissionTest.csv",prediction)

"sampleSubmissionTest.csv"